Implementing the tutorial provided but withastropy fits as the fits reader

Installing packages

In [ ]:
!pip install astropy
!pip install sep

Setting up imports for data visualization

In [ ]:
import numpy as np
import sep

In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams

#using inbuilt magic functions
%matplotlib inline

rcParams['figure.figsize'] = [10.,8.]


Reading in image and data

In [ ]:
data = fits.open("image.fits")[0].data
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();

# saving image as instructed on canvas
plt.savefig('firstRender - tutorial.png')


BACK GROUND SUBTRACTION


In [ ]:
bkg = sep.Background(data)

In [ ]:
print(bkg.globalback)
print(bkg.globalrms)

#prints 6852.07 and 65.612 on local machine


In [ ]:
bkg_image = bkg.back()




In [ ]:
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

# saving image
plt.savefig('render-2-tutorial.png')



In [ ]:
bkg_rms = bkg.rms()


In [ ]:
# show the background noise
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

#saving background noise render image
plt.savefig('render-3-background-noise.png')

HERE IS WHERE WE SUBTRACT THE BACKGROUND

In [ ]:
data_sub = data-bkg


#can also do so with bkg.subfrom(data)



OBJECT DETECTION


In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

In [ ]:
# how many objects were detected
len(objects)

In [ ]:
from matplotlib.patches import Ellipse

# plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

# plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

plt.savefig('render4-objectplot.png')

In [ ]:
# available fields
objects.dtype.names

#on local machine - output - ('thresh','npix','tnpix','xmin','xmax''ymin','ymax','x','y','x2','y2','xy','errx2','erry2','errxy','a','b','theta','cxx','cyy','cxy','cflux','flux','cpeak','peak''xcpeak','ycpeak','xpeak','ypeak',

APERTURE PHOTOMETRY
Here we perform simple circular aperture photometry with a 3 pixel radius at the locations of the objects:


In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)


In [ ]:
# show the first 10 objects results:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))